##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recommending Movies: Recommender Models in TFX

Note: We recommend running this tutorial in a Colab notebook, with no setup
required!  Just click "Run in Google Colab".

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/recommenders">
<img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/recommenders.ipynb">
<img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/recommenders.ipynb">
<img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/recommenders.ipynb">
<img width=32px src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a></td>
</table></div>

## TFRS Tutorial Ported to TFX
This is a port of a basic TensorFlow Recommenders (TFRS) tutorial to TFX, which is designed to demonstrate how to use TFRS in a TFX pipeline.  It mirrors the [basic tutorial](https://www.tensorflow.org/recommenders/examples/basic_retrieval).

For context, real-world recommender systems are often composed of two stages:

1. The retrieval stage is responsible for selecting an initial set of hundreds of candidates from all possible candidates. The main objective of this model is to efficiently weed out all candidates that the user is not interested in. Because the retrieval model may be dealing with millions of candidates, it has to be computationally efficient.
2. The ranking stage takes the outputs of the retrieval model and fine-tunes them to select the best possible handful of recommendations. Its task is to narrow down the set of items the user may be interested in to a shortlist of likely candidates.

In this tutorial, we're going to focus on the first stage, retrieval. Retrieval models are often composed of two sub-models:

1. A query model computing the query representation (normally a fixed-dimensionality embedding vector) using query features.
2. A candidate model computing the candidate representation (an equally-sized vector) using the candidate features

The outputs of the two models are then multiplied together to give a query-candidate affinity score, with higher scores expressing a better match between the candidate and the query.

In this tutorial, we're going to build and train such a two-tower model using the Movielens dataset.

We're going to:

1. Ingest and inspect the MovieLens dataset.
2. Implement a retrieval model.
3. Train and export the model.
4. Make predictions

## The dataset

The Movielens dataset is a classic dataset from the [GroupLens](https://grouplens.org/datasets/movielens/) research group at the University of Minnesota. It contains a set of ratings given to movies by a set of users, and is a workhorse of recommender system research.

The data can be treated in two ways:

1. It can be interpreted as expressesing which movies the users watched (and rated), and which they did not. This is a form of implicit feedback, where users' watches tell us which things they prefer to see and which they'd rather not see.
2. It can also be seen as expressesing how much the users liked the movies they did watch. This is a form of explicit feedback: given that a user watched a movie, we can tell roughly how much they liked by looking at the rating they have given.

In this tutorial, we are focusing on a retrieval system: a model that predicts a set of movies from the catalogue that the user is likely to watch. Often, implicit data is more useful here, and so we are going to treat Movielens as an implicit system. This means that every movie a user watched is a positive example, and every movie they have not seen is an implicit negative example.

## Imports


Let's first get our imports out of the way.

### Uninstall shapely

TODO(b/263441833) This is a temporal solution to avoid an
ImportError. Ultimately, it should be handled by supporting a
recent version of Bigquery, instead of uninstalling other extra
dependencies.

### Did you restart the runtime?
If you are using Google Colab, the first time that you run the cell above, you must restart the runtime (Runtime > Restart runtime ...). This is because of the way that Colab loads packages.

In [7]:
import pymysql
from sqlalchemy import create_engine

In [16]:
# ENV
engine = create_engine('mysql+pymysql://?:?@ ?/?', convert_unicode=False) 

In [17]:
import pandas as pd

query = 'SELECT a.user_id, a.food_name, a.date, c.age, c.gender FROM user_meal_table As a LEFT JOIN ingredient_table as b on a.food_code = b.food_code LEFT JOIN user_table as c on a.user_id = c.user_id'
df = pd.read_sql_query(query, engine)

In [18]:
df

,user_id,food_name,date,age,gender
0,test1,항정살,2022-08-31,23,m
1,test1,밥,2022-08-31,23,m
2,test1,밥,2023-02-20,23,m
3,test1,깍두기,2022-09-01,23,m
4,test1,쌀밥,2022-09-01,23,m
5,test1,삼겹살,2022-09-01,23,m
6,mintaehee,김치만두,2022-09-01,26,m
7,mintaehee,쌀밥,2022-02-21,26,m
8,mintaehee,"라면, 비빔라면, 끓인것",2023-02-21,26,m
9,mintaehee,쌀밥,2023-02-20,26,m


In [26]:
df.to_csv(r'./rating/ratings.CSV', index=False, encoding='utf-8')
pd.DataFrame({"food_name":df['food_name'].unique().tolist()}).to_csv(r'./food/foods.CSV', index=False, encoding='utf-8')

In [2]:
import os
import absl
import json
import pprint
import tempfile

from typing import Any, Dict, List, Text

import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import apache_beam as beam

from absl import logging

from tfx.components.example_gen.base_example_gen_executor import BaseExampleGenExecutor
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.components.example_gen import utils
from tfx.dsl.components.base import executor_spec

from tfx.types import artifact
from tfx.types import artifact_utils
from tfx.types import channel
from tfx.types import standard_artifacts
from tfx.types.standard_artifacts import Examples

from tfx.dsl.component.experimental.annotations import InputArtifact
from tfx.dsl.component.experimental.annotations import OutputArtifact
from tfx.dsl.component.experimental.annotations import Parameter
from tfx.dsl.component.experimental.decorators import component
from tfx.types.experimental.simple_artifacts import Dataset

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Set up logging.
tf.get_logger().propagate = False
absl.logging.set_verbosity(absl.logging.INFO)
pp = pprint.PrettyPrinter()

print(f"TensorFlow version: {tf.__version__}")
print(f"TFX version: {tfx.__version__}")
print(f"TensorFlow Recommenders version: {tfrs.__version__}")

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

TensorFlow version: 2.11.0
TFX version: 1.12.0
TensorFlow Recommenders version: v0.7.3
The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


## Create a TFDS ExampleGen

We create a [custom ExampleGen component](https://www.tensorflow.org/tfx/guide/examplegen#custom_examplegen) which we use to load a TensorFlow Datasets (TFDS) dataset.  This uses a custom executor in a FileBasedExampleGen.

## Init TFX Pipeline Context

In [24]:
context = InteractiveContext()

## Preparing the dataset

We will use our custom executor in a `FileBasedExampleGen` to load our datasets from TFDS.  Since we have two datasets, we will create two `ExampleGen` components.

import CsvExampleGen from tfx

In [29]:
ratings_example_gen = tfx.components.CsvExampleGen(input_base='./rating')
context.run(ratings_example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data ./rating/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 4
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [30]:
from  tfx.proto import example_gen_pb2

# Input has a single split 'input_dir/*'.
# Output 2 splits based on 'user_id' features: train:eval=3:1.
output = example_gen_pb2.Output(
             split_config=example_gen_pb2.SplitConfig(splits=[
                 example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=3),
             ],
             partition_feature_name="food_name"))
foods_example_gen = tfx.components.CsvExampleGen(input_base='./food',output_config=output)
context.run(foods_example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data ./food/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 5
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Create `inspect_examples` utility
We create a convenience utility to inspect datasets of TF.Examples. The ratings dataset returns a dictionary of movie id, user id, the assigned rating, timestamp, movie information, and user information:

In [31]:
def inspect_examples(component,
                     channel_name='examples',
                     split_name='train',
                     num_examples=1):
  # Get the URI of the output artifact, which is a directory
  full_split_name = 'Split-{}'.format(split_name)
  print('channel_name: {}, split_name: {} (\"{}\"), num_examples: {}\n'.format(
      channel_name, split_name, full_split_name, num_examples))
  train_uri = os.path.join(
      component.outputs[channel_name].get()[0].uri, full_split_name)

  # Get the list of files in this directory (all compressed TFRecord files)
  tfrecord_filenames = [os.path.join(train_uri, name)
                        for name in os.listdir(train_uri)]

  # Create a `TFRecordDataset` to read these files
  dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

  # Iterate over the records and print them
  for tfrecord in dataset.take(num_examples):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

inspect_examples(ratings_example_gen)

channel_name: examples, split_name: train ("Split-train"), num_examples: 1

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 23
      }
    }
  }
  feature {
    key: "date"
    value {
      bytes_list {
        value: "2022-08-31"
      }
    }
  }
  feature {
    key: "food_name"
    value {
      bytes_list {
        value: "\353\260\245"
      }
    }
  }
  feature {
    key: "gender"
    value {
      bytes_list {
        value: "m"
      }
    }
  }
  feature {
    key: "user_id"
    value {
      bytes_list {
        value: "test1"
      }
    }
  }
}



2023-02-23 16:09:47.311336: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-23 16:09:47.311380: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-23 16:09:47.311402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (0978014ebea8): /proc/driver/nvidia/version does not exist
2023-02-23 16:09:47.311687: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [32]:
inspect_examples(foods_example_gen)

channel_name: examples, split_name: train ("Split-train"), num_examples: 1

features {
  feature {
    key: "food_name"
    value {
      bytes_list {
        value: "\355\225\255\354\240\225\354\202\264"
      }
    }
  }
}



The movies dataset contains the movie id, movie title, and data on what genres it belongs to. Note that the genres are encoded with integer labels.

## ExampleGen did the split
When we ingested the movie lens dataset, our `ExampleGen` component split the data into `train` and `eval` splits.  They are actually named `Split-train` and `Split-eval`. By default the split is 66% training, 34% evaluation.

## Generate statistics for movies and ratings

For a TFX pipeline we need to generate statistics for the dataset.  We do that by using a [StatisticsGen component](https://www.tensorflow.org/tfx/guide/statsgen). These will be used by the [SchemaGen component](https://www.tensorflow.org/tfx/guide/schemagen) below when we generate a schema for our dataset.  This is good practice anyway, because it's important to examine and analyze your data on an ongoing basis.  Since we have two datasets we will create two StatisticsGen components.

In [33]:
ratings_stats_gen = tfx.components.StatisticsGen(
    examples=ratings_example_gen.outputs['examples']
)

context.run(ratings_stats_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/StatisticsGen/statistics/6/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/StatisticsGen/statistics/6/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 6
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [34]:
context.show(ratings_stats_gen.outputs['statistics'])

In [35]:
foods_stats_gen = tfx.components.StatisticsGen(
    examples=foods_example_gen.outputs['examples'])
context.run(foods_stats_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/StatisticsGen/statistics/7/Split-train.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 7
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [36]:
context.show(foods_stats_gen.outputs['statistics'])

## Create schemas for movies and ratings

For a TFX pipeline we need to generate a data schema from our dataset.  We do that by using a [SchemaGen component](https://www.tensorflow.org/tfx/guide/schemagen). This will be used by the [Transform component](https://www.tensorflow.org/tfx/guide/transform) below to do our feature engineering in a way that is highly scalable to large datasets, and avoids training/serving skew.  Since we have two datasets we will create two SchemaGen components.

In [37]:
ratings_schema_gen = tfx.components.SchemaGen(
    statistics=ratings_stats_gen.outputs['statistics'],
    infer_feature_shape=False
)
context.run(ratings_schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/SchemaGen/schema/8/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 8
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [38]:
context.show(ratings_schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,single,-
'date',STRING,required,single,'date'
'food_name',STRING,required,single,'food_name'
'gender',STRING,required,single,'gender'
'user_id',STRING,required,single,'user_id'


,Values
Domain,
'date',"'2022-02-21', '2022-08-31', '2022-09-01', '2023-02-20', '2023-02-21', '2023-02-22', '2023-02-23', '2023-02-24'"
'food_name',"'밥', '삼겹살', '생선구이.json', '스파게티', '안심스테이크', '궁중떡볶이', '김치찌개', '깍두기', '덮밥, 불고기 덮밥', '된장, 개량, 찌개용', '떡볶이', '라면, 비빔라면, 끓인것', '밥', '불고기피자', '삼겹살', '소주, 알코올 17.8%', '쌀밥', '알밥', '항정살', '김치만두', '담치, 진주담치(양식), 생것', '돌산갓김치', '된장찌개', '츄러스', '컵라면'"
'gender',"'f', 'm'"
'user_id',"'kimsiyu', 'mintaehee', 'test1'"


In [39]:
foods_schema_gen = tfx.components.SchemaGen(
    statistics=foods_stats_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(foods_schema_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Schema written to /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/SchemaGen/schema/9/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 9
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [40]:
context.show(foods_schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'food_name',STRING,required,single,'food_name'


,Values
Domain,
'food_name',"'밥', '삼겹살', '생선구이.json', '스파게티', '안심스테이크', '궁중떡볶이', '김치만두', '김치찌개', '깍두기', '담치, 진주담치(양식), 생것', '덮밥, 불고기 덮밥', '돌산갓김치', '된장, 개량, 찌개용', '된장찌개', '떡볶이', '라면, 비빔라면, 끓인것', '밥', '불고기피자', '삼겹살', '소주, 알코올 17.8%', '쌀밥', '알밥', '츄러스', '컵라면', '항정살'"


## Feature Engineering using Transform

For a structured and repeatable design of a TFX pipeline we will need a scalable approach to feature engineering.  This allows us to handle the large datasets which are usually part of many recommender systems, and it also avoids training/serving skew.  We will do that using the [Transform component](https://www.tensorflow.org/tfx/guide/transform).

The Transform component uses a module file to supply user code for the feature engineering what we want to do, so our first step is to create that module file. Since we have two datasets, we will create two of these module files and two Transform components.

One of the things that our recommender needs is vocabularies for the `user_id` and `movie_title` fields.  In the [basic_retrieval tutorial](https://www.tensorflow.org/recommenders/examples/basic_retrieval) those are created with inline Numpy, but here we will use Transform.

Note: The `%%writefile {_movies_transform_module_file}` cell magic below creates and writes the contents of that cell to a file on the notebook server where this notebook is running (for example, the Colab VM).  When doing this outside of a notebook you would just create a Python file.

In [41]:
_foods_transform_module_file = 'foods_transform_module.py'

In [42]:
%%writefile {_foods_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

def preprocessing_fn(inputs):
  # We only want the movie title
  return {'food_name':inputs['food_name']}

Writing foods_transform_module.py


In [43]:
foods_transform = tfx.components.Transform(
    examples=foods_example_gen.outputs['examples'],
    schema=foods_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_foods_transform_module_file))
context.run(foods_transform, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for '/home/jovyan/work/foods_transform_module.py' (including modules: ['foods_transform_module']).
INFO:absl:User module package has hash fingerprint version f934d36a48198fea8257945eef3aec0f38131ae616e3de4ed531a57be75acbf9.
INFO:absl:Executing: ['/opt/conda/bin/python', '/tmp/tmplspbxiya/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpuhxg8w5s', '--dist-dir', '/tmp/tmpa7lknnf5']
/opt/conda/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+f934d36a48198fea8257945eef3aec0f38131ae616e3de4ed531a57be75acbf9-py3-none-any.whl'; target user module is 'foods_transform_module'.
INFO:absl:Full user module path is 'foods_transform_module@/tmp/tf

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying foods_transform_module.py -> build/lib
installing to /tmp/tmpuhxg8w5s
running install
running install_lib
copying build/lib/foods_transform_module.py -> /tmp/tmpuhxg8w5s
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpuhxg8w5s/tfx_user_code_Transform-0.0+f934d36a48198fea8257945eef3aec0f38131ae616e3de4ed531a57be75acbf9-py3.8.egg-info
running install_scripts
creating /tmp/tmpuhxg8w5s/tfx_user_code_Transfo

INFO:absl:Successfully installed '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+f934d36a48198fea8257945eef3aec0f38131ae616e3de4ed531a57be75acbf9-py3-none-any.whl'.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'foods_transform_module@/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+f934d36a48198fea8257945eef3aec0f38131ae616e3de4ed531a57be75acbf9-py3-none-any.whl', 'stats_options_updater_fn': None} 'stats_options_updater_fn'
INFO:absl:Installing '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+f934d36a48198fea8257945eef3aec0f38131ae616e3de4ed531a57be75acbf9-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/opt/conda/bin/python', '-m', 'pip', 'install', '--target', '/tmp/tmpk2ckr7d8', '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+f934d36a48198fea8257945eef3aec0f38131ae616

Processing /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+f934d36a48198fea8257945eef3aec0f38131ae616e3de4ed531a57be75acbf9-py3-none-any.whl


INFO:absl:Successfully installed '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+f934d36a48198fea8257945eef3aec0f38131ae616e3de4ed531a57be75acbf9-py3-none-any.whl'.
INFO:absl:Installing '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+f934d36a48198fea8257945eef3aec0f38131ae616e3de4ed531a57be75acbf9-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/opt/conda/bin/python', '-m', 'pip', 'install', '--target', '/tmp/tmpyj3jg8em', '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+f934d36a48198fea8257945eef3aec0f38131ae616e3de4ed531a57be75acbf9-py3-none-any.whl']


...

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/Transform/transform_graph/10/.temp_path/tftransform_tmp/9f86c6c5d32e49178ec9cf8cbaa29aa7/assets


INFO:absl:Feature food_name has no shape. Setting to VarLenSparseTensor.


INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.


INFO:absl:Feature food_name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature food_name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature food_name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature food_name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature food_name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature food_name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 10
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [44]:
context.show(foods_transform.outputs['post_transform_schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'food_name',BYTES,,,-


In [45]:
inspect_examples(foods_transform, channel_name='transformed_examples')

channel_name: transformed_examples, split_name: train ("Split-train"), num_examples: 1

features {
  feature {
    key: "food_name"
    value {
      bytes_list {
        value: "\355\225\255\354\240\225\354\202\264"
      }
    }
  }
}



In [46]:
_ratings_transform_module_file = 'ratings_transform_module.py'

In [47]:
%%writefile {_ratings_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft
import pdb
import numpy as np
from typing import Dict, List, Text, Any

NUM_OOV_BUCKETS = 1

def preprocessing_fn(inputs: Dict[Text, Any]) -> Dict[Text, Any]:
  # We only want the user ID and the movie title, but we also need vocabularies
  # for both of them.  The vocabularies aren't features, they're only used by
  # the lookup.
  outputs = {}
  # print(inputs['user_id'])
  # outputs['user_id'] = tf.transpose(tft.sparse_tensor_to_dense_with_shape(inputs['user_id'], [None, 1], '-1'))
  # outputs['food_name'] = tf.transpose(tft.sparse_tensor_to_dense_with_shape(inputs['food_name'], [None, 1], '-1'))
  # outputs['gender'] =tf.transpose(tft.sparse_tensor_to_dense_with_shape(inputs['gender'], [None, 1], '-1'))
  # signature_dict = { "user_id": tf.TensorSpec(shape=(None,), dtype=tf.string, name="user_id"),
  #                  "gender": tf.TensorSpec(shape=(None,), dtype=tf.string, name="gender") }
  outputs['user_id'] = tft.sparse_tensor_to_dense_with_shape(inputs['user_id'], [None, 1], '-1')
  outputs['food_name'] = tft.sparse_tensor_to_dense_with_shape(inputs['food_name'], [None, 1], '-1')
  outputs['gender'] = tft.sparse_tensor_to_dense_with_shape(inputs['gender'], [None, 1], '-1')
  outputs['age'] = tft.sparse_tensor_to_dense_with_shape(inputs['age'], [None, 1], -1)

  tft.compute_and_apply_vocabulary(
      inputs['user_id'],
      num_oov_buckets=NUM_OOV_BUCKETS,
      vocab_filename='user_id_vocab')

  tft.compute_and_apply_vocabulary(
      inputs['food_name'],
      num_oov_buckets=NUM_OOV_BUCKETS,
      vocab_filename='food_name_vocab')

  tft.compute_and_apply_vocabulary(
    inputs['gender'],
    num_oov_buckets=NUM_OOV_BUCKETS,
    vocab_filename='gender_name_vocab')

  return outputs

Writing ratings_transform_module.py


In [48]:
ratings_transform = tfx.components.Transform(
    examples=ratings_example_gen.outputs['examples'],
    schema=ratings_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_ratings_transform_module_file),
  )

context.run(ratings_transform, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for '/home/jovyan/work/ratings_transform_module.py' (including modules: ['ratings_transform_module', 'foods_transform_module']).
INFO:absl:User module package has hash fingerprint version 2364946c189b10e8e7ffad0d517780c689d0ca5d5a7570a461ec03398b1d3707.
INFO:absl:Executing: ['/opt/conda/bin/python', '/tmp/tmprrqr29zz/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpblzivbmg', '--dist-dir', '/tmp/tmphp76k6rd']
/opt/conda/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+2364946c189b10e8e7ffad0d517780c689d0ca5d5a7570a461ec03398b1d3707-py3-none-any.whl'; target user module is 'ratings_transform_module'.
INFO:absl:Full user module path is

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying ratings_transform_module.py -> build/lib
copying foods_transform_module.py -> build/lib
installing to /tmp/tmpblzivbmg
running install
running install_lib
copying build/lib/ratings_transform_module.py -> /tmp/tmpblzivbmg
copying build/lib/foods_transform_module.py -> /tmp/tmpblzivbmg
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpblzivbmg/tfx_user_code_Transform-0.0+2364946c189b10e8e7ffad0d517780c689d0

INFO:absl:Successfully installed '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+2364946c189b10e8e7ffad0d517780c689d0ca5d5a7570a461ec03398b1d3707-py3-none-any.whl'.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'ratings_transform_module@/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+2364946c189b10e8e7ffad0d517780c689d0ca5d5a7570a461ec03398b1d3707-py3-none-any.whl', 'stats_options_updater_fn': None} 'stats_options_updater_fn'
INFO:absl:Installing '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+2364946c189b10e8e7ffad0d517780c689d0ca5d5a7570a461ec03398b1d3707-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/opt/conda/bin/python', '-m', 'pip', 'install', '--target', '/tmp/tmpq0yr69b5', '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+2364946c189b10e8e7ffad0d517780c689d0ca5d

Processing /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+2364946c189b10e8e7ffad0d517780c689d0ca5d5a7570a461ec03398b1d3707-py3-none-any.whl


INFO:absl:Successfully installed '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+2364946c189b10e8e7ffad0d517780c689d0ca5d5a7570a461ec03398b1d3707-py3-none-any.whl'.
INFO:absl:Installing '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+2364946c189b10e8e7ffad0d517780c689d0ca5d5a7570a461ec03398b1d3707-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/opt/conda/bin/python', '-m', 'pip', 'install', '--target', '/tmp/tmpjzy11i59', '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Transform-0.0+2364946c189b10e8e7ffad0d517780c689d0ca5d5a7570a461ec03398b1d3707-py3-none-any.whl']


...

INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.


INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature food_name has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature food_name has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature food_name has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has

INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.


INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature food_name has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature food_name has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature food_name has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has

ExecutionResult(
    component_id: Transform
    execution_id: 11
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [49]:
context.show(ratings_transform.outputs['post_transform_schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'food_name',BYTES,required,,-
'gender',BYTES,required,,-
'user_id',BYTES,required,,-


In [50]:
inspect_examples(ratings_transform, channel_name='transformed_examples')

channel_name: transformed_examples, split_name: train ("Split-train"), num_examples: 1

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 23
      }
    }
  }
  feature {
    key: "food_name"
    value {
      bytes_list {
        value: "\353\260\245"
      }
    }
  }
  feature {
    key: "gender"
    value {
      bytes_list {
        value: "m"
      }
    }
  }
  feature {
    key: "user_id"
    value {
      bytes_list {
        value: "test1"
      }
    }
  }
}



## Implementing a model in TFX

In the [basic_retrieval](https://www.tensorflow.org/recommenders/examples/basic_retrieval) tutorial the model was created inline in the Python runtime. In a TFX pipeline, the model, metric, and loss are defined and trained in the module file for a [pipeline component called Trainer](https://www.tensorflow.org/tfx/guide/trainer). This makes the model, metric, and loss part of a repeatable process which can be automated and monitored.

### TensorFlow Recommenders model architecture

We are going to build a two-tower retrieval model. The concept of two-tower means we will have a query tower computing the user representation using user features, and another item tower computing the movie representation using the movie features. We can build each tower separately (in the `_build_user_model()` and `_build_movie_model()` methods below) and then combine them in the final model (as in the `MobieLensModel` class). `MovieLensModel` is a subclass of `tfrs.Model` base class, which streamlines building models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value.

In [51]:
# We're now going to create the module file for Trainer, which will include the
# code above with some modifications for TFX.

_trainer_module_file = 'trainer_module.py'

In [52]:
%%writefile {_trainer_module_file}

from typing import Dict, List, Text

import pdb

import os
import absl
import datetime
import glob
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_recommenders as tfrs

from absl import logging
from tfx.types import artifact_utils

from tfx import v1 as tfx
from tfx_bsl.coders import example_coder
from tfx_bsl.public import tfxio

import numpy as np

layer_sizes = [64,32]
INPUT_FN_BATCH_SIZE = 1

def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  try:
    return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size),
      tf_transform_output.transformed_metadata.schema)
  except BaseException as err:
    logging.error('######## ERROR IN _input_fn:\n{}\n###############'.format(err))

  return None

def extract_str_feature(dataset, feature_name):
  np_dataset = []
  for example in dataset:
    np_example = example_coder.ExampleToNumpyDict(example.numpy())
    np_dataset.append(np_example[feature_name][0].decode())
  return tf.data.Dataset.from_tensor_slices(np_dataset)



class UserModel(tf.keras.Model):
  
  def __init__(self,tf_transform_output, movies_uri):
    super().__init__()

    unique_user_ids = tf_transform_output.vocabulary_by_name('user_id_vocab')
    users_vocab_str = [b.decode() for b in unique_user_ids]

    unique_genders = tf_transform_output.vocabulary_by_name('gender_name_vocab')
    genders_vocab_str = [b.decode() for b in unique_genders]

    age_buckets = np.linspace(
        0, 100, num=10,
    )
    # unique_food_names = np.unique(np.concatenate(list(foods.batch(1000))))
    # unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    #     lambda x: x["user_id"]))))
    # unique_genders = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    #     lambda x: x["gender"]))))

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])
    self.gender_embedding = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_genders, mask_token=None),
    tf.keras.layers.Embedding(len(unique_genders) + 1, 32),
    ])

    self.age_embedding = tf.keras.Sequential([
        tf.keras.layers.Discretization(age_buckets.tolist()),
        tf.keras.layers.Embedding(len(age_buckets) + 1, 32),
    ])

    # ages_artifact = movies_uri.get()[0]
    # input_dir = artifact_utils.get_split_uri([ages_artifact], 'train')
    # age_files = glob.glob(os.path.join(input_dir, '*'))
    # ages = tf.data.TFRecordDataset(age_files, compression_type="GZIP")
    # ages_dataset = extract_str_feature(ages, 'ages')
    # self.normalized_age.adapt(ages_dataset)


  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    # print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    # print(tf.transpose(inputs["user_id"].shape))
    # print(tf.transpose(inputs["gender"].shape))
    # print(inputs["user_id"].shape)
    # print(inputs["gender"].shape)
    # print(self.user_embedding(inputs["user_id"]).shape)
    # print(self.user_embedding(inputs["gender"]).shape)
    # print(tf.concat([
    #     tf.transpose(self.user_embedding(inputs["user_id"])),
    #     tf.transpose(self.gender_embedding(tf.transpose(inputs["gender"])))
    # ], axis=0).shape)

    # print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.")
    # if len(dim) == 1:
    t1 = tf.squeeze(inputs["user_id"])
    t1 = self.user_embedding(t1)
    t1 = tf.reshape(t1, shape=[-1,32])

    t2 = tf.squeeze(inputs['gender'])
    t2 = self.gender_embedding(t2)
    t2 = tf.reshape(t2,shape=[-1,32])

    t3 = tf.squeeze(inputs["age"])
    t3 = self.age_embedding(t3)
    t3 = tf.reshape(t3, shape=[-1,32])
    return tf.concat([
        t1,
        t2,
        t3
    ], axis=1)
    # return tf.concat([
    #     tf.reshape(self.user_embedding(tf.squeeze(inputs["user_id"])),shape=[-1,32]),
    #     tf.reshape(self.gender_embedding(tf.squeeze(inputs["gender"])),shape=[-1,32])
    # ], axis=1)
    # else:
    #   return tf.concat([
    #       self.user_embedding(tf.squeeze(inputs["user_id"])),
    #       self.gender_embedding(tf.squeeze(inputs["gender"]))
    #   ], axis=1)

class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, tf_transform_output, movies_uri):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()
    # We first use the user model for generating embeddings.
    self.embedding_model = UserModel(tf_transform_output, movies_uri)

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))
    
  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

class MovieModel(tf.keras.Model):
  
  def __init__(self, tf_transform_output, movies_uri):
    super().__init__()

    max_tokens = 10_000

    unique_food_names = tf_transform_output.vocabulary_by_name('food_name_vocab')
    foods_vocab_str = [b.decode() for b in unique_food_names]

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=foods_vocab_str,mask_token=None),
      tf.keras.layers.Embedding(len(foods_vocab_str) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])
    # 용의자


    movies_artifact = movies_uri.get()[0]
    input_dir = artifact_utils.get_split_uri([movies_artifact], 'train')
    movie_files = glob.glob(os.path.join(input_dir, '*'))
    movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")
    movies_dataset = extract_str_feature(movies, 'food_name')

    self.title_vectorizer.adapt(movies_dataset)

    # self.title_vectorizer.adapt(foods)

  def call(self, foods):
    # print("------------------------")
    # print(self.title_embedding(foods).shape)
    # print(self.title_text_embedding(foods).shape)
    # print(tf.concat([
    #     self.title_embedding(foods),
    #     self.title_text_embedding(foods),
    # ], axis=1).shape)
    # print("----------------------------")
    return tf.concat([
        self.title_embedding(foods),
        self.title_text_embedding(foods),
    ], axis=1)

class CandidateModel(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, tf_transform_output, movies_uri):
    """Model for encoding movies.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.embedding_model = MovieModel(tf_transform_output, movies_uri)

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()


    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))
    
  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)


class MovielensModel(tfrs.models.Model):

  def __init__(self, tf_transform_output, movies_uri):
    super().__init__()
    self.query_model = QueryModel(tf_transform_output, movies_uri)
    self.candidate_model = CandidateModel(tf_transform_output, movies_uri)

    movies_artifact = movies_uri.get()[0]
    input_dir = artifact_utils.get_split_uri([movies_artifact], 'train')
    movie_files = glob.glob(os.path.join(input_dir, '*'))
    movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")
    movies_dataset = extract_str_feature(movies, 'food_name')
    print("movies_dataset >>>>>", movies_dataset.batch(128).map(self.candidate_model))
    for i in movies_dataset:
      print("아아아",i)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies_dataset.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "gender": features["gender"],
        "age": features["age"]
    })
    movie_embeddings = self.candidate_model(features["food_name"])

    return self.task(
        query_embeddings, movie_embeddings, compute_metrics=not training)

class goruf(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, index):
    super().__init__()

    self.index = index
    
  def call(self, inputs):
    input = {}
    input['user_id'] = tf.reshape(inputs['user_id'], shape=[2,1])
    input['gender'] = tf.reshape(inputs['gender'], shape=[2,1])
    input['age'] = tf.reshape(inputs['age'], shape=[2,1])
    self.tl = self.index(input)
    return self.tl


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""
  try:
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
      """Returns the output to be used in the serving signature."""
      try:
        feature_spec = tf_transform_output.raw_feature_spec()
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        print("transformed_features >>>> ",transformed_features)
        result1 = model(transformed_features)
      except BaseException as err:
        logging.error('######## ERROR IN serve_tf_examples_fn:\n{}\n###############'.format(err))
      return result1
  except BaseException as err:
      logging.error('######## ERROR IN _get_serve_tf_examples_fn:\n{}\n###############'.format(err))

  return serve_tf_examples_fn
  
# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  try:
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor,
                              tf_transform_output, INPUT_FN_BATCH_SIZE)
    eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor,
                            tf_transform_output, INPUT_FN_BATCH_SIZE)

    model = MovielensModel(
        tf_transform_output,
        fn_args.custom_config['foods']
        )

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=fn_args.model_run_dir, update_freq='batch')

    model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

  except BaseException as err:
    logging.error('######## ERROR IN run_fn before fit:\n{}\n###############'.format(err))

  try:
    # model.fit(
    #     train_dataset,
    #     validation_data=eval_dataset,
    #     validation_freq=5,
    #     epochs=fn_args.custom_config['epochs'],
    #     steps_per_epoch=fn_args.train_steps,
    #     verbose=0)
    model.fit(
        train_dataset,
        epochs=fn_args.custom_config['epochs'],
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback])

      # model.fit(
      # train_dataset,
      # validation_data=eval_dataset,
      # validation_freq=5,
      # steps_per_epoch=1,
      # epochs=5,
      # callbacks=[tensorboard_callback],
      # verbose=1)
    print(model.evaluate(eval_dataset, steps = fn_args.eval_steps, return_dict=True))
  except BaseException as err:
      logging.error('######## ERROR IN run_fn during fit:\n{}\n###############'.format(err))

  try:

    index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)
    movies_artifact = fn_args.custom_config['foods'].get()[0]
    input_dir = artifact_utils.get_split_uri([movies_artifact], 'train')
    movie_files = glob.glob(os.path.join(input_dir, '*'))
    movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")
    # 의심이 됨
    movies_dataset = extract_str_feature(movies, 'food_name')

    index.index_from_dataset(
      tf.data.Dataset.zip((
          movies_dataset.batch(100),
          movies_dataset.batch(100).map(model.candidate_model))
      )
    )

    _, titles = index({'user_id':tf.constant(["test1","mintaehee"]),'gender':tf.constant(["m","m"]),"age":tf.constant([23,26])})
    print("titlse >>>",titles)
    # Run once so that we can get the right signatures into SavedModel
    # _, titles = index(tf.constant(["42"]))
    # print(f"Recommendations for user 42: {titles[0, :3]}")
    # signature_dict = { "user_id": tf.TensorSpec(shape=(None,), dtype=tf.string, name="user_id"),
    #                "gender": tf.TensorSpec(shape=(None,), dtype=tf.string, name="gender") }
    # signatures = {
    #     'serving_default':
    #         _get_serve_tf_examples_fn(index,
    #                                   tf_transform_output).get_concrete_function(
    #                                       tf.TensorSpec(
    #                                           shape=[None],
    #                                           dtype=tf.string,
    #                                           name='examples'),),
    # }

    # signatures = {
    #     'serving_default':
    #         _get_serve_tf_examples_fn(index,
    #                                   tf_transform_output).get_concrete_function(
    #                                       signature_dict),
    # }
    

    new_model = goruf(index)
    # index.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)
    _ , titles = new_model({'user_id':tf.constant(["test1","mintaehee"]),'gender':tf.constant(["m","m"]),"age":tf.constant([23,26])})
    # print(titles)
    new_model.save(fn_args.serving_model_dir,save_format='tf')
    # index.save(fn_args.serving_model_dir, save_format='tf')
    # index.save(fn_args.serving_model_dir, save_format='tf')
  except BaseException as err:
      logging.error('######## ERROR IN run_fn during export:\n{}\n###############'.format(err))



Writing trainer_module.py


In [53]:
import glob
movies_artifact = ratings_transform.outputs['transformed_examples'].get()[0]
input_dir = artifact_utils.get_split_uri([movies_artifact], 'train')
movie_files = glob.glob(os.path.join(input_dir, '*'))
movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")
movies
# movies_dataset = extract_str_feature(movies, 'food_name')
# movies_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [54]:
data = movies.__iter__()
next(data)

<tf.Tensor: shape=(), dtype=string, numpy=b'\nK\n\x14\n\tfood_name\x12\x07\n\x05\n\x03\xeb\xb0\xa5\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01\x17\n\x14\n\x07user_id\x12\t\n\x07\n\x05test1\n\x0f\n\x06gender\x12\x05\n\x03\n\x01m'>

## Training the model

After defining the model, we can run the [Trainer component](https://www.tensorflow.org/tfx/guide/trainer) to do the model training.

In [55]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_trainer_module_file),
    examples=ratings_transform.outputs['transformed_examples'],
    transform_graph=ratings_transform.outputs['transform_graph'],
    schema=ratings_transform.outputs['post_transform_schema'],
    train_args=tfx.proto.TrainArgs(num_steps=100),
    eval_args=tfx.proto.EvalArgs(num_steps=5),
    custom_config={
        'epochs':5,
        'foods':foods_transform.outputs['transformed_examples'],
        'food_schema':foods_transform.outputs['post_transform_schema'],
        'ratings':ratings_transform.outputs['transformed_examples'],
        'ratings_schema':ratings_transform.outputs['post_transform_schema']
        })


context.run(trainer, enable_cache=False)

INFO:absl:Generating ephemeral wheel package for '/home/jovyan/work/trainer_module.py' (including modules: ['trainer_module', 'ratings_transform_module', 'foods_transform_module']).
INFO:absl:User module package has hash fingerprint version 3e183ecd1f1b457a1f72ad9f7e5d977cdfdc776b5fb343307dc55e147e1e7263.
INFO:absl:Executing: ['/opt/conda/bin/python', '/tmp/tmpeguprpvi/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp73zxf81q', '--dist-dir', '/tmp/tmpz1kn7na9']
/opt/conda/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Trainer-0.0+3e183ecd1f1b457a1f72ad9f7e5d977cdfdc776b5fb343307dc55e147e1e7263-py3-none-any.whl'; target user module is 'trainer_module'.
INFO:absl:Full user module path is 'tr

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer_module.py -> build/lib
copying ratings_transform_module.py -> build/lib
copying foods_transform_module.py -> build/lib
installing to /tmp/tmp73zxf81q
running install
running install_lib
copying build/lib/trainer_module.py -> /tmp/tmp73zxf81q
copying build/lib/ratings_transform_module.py -> /tmp/tmp73zxf81q
copying build/lib/foods_transform_module.py -> /tmp/tmp73zxf81q
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /t

INFO:absl:Successfully installed '/tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/_wheels/tfx_user_code_Trainer-0.0+3e183ecd1f1b457a1f72ad9f7e5d977cdfdc776b5fb343307dc55e147e1e7263-py3-none-any.whl'.
INFO:absl:Training model.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature food_name has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature food_name has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.


movies_dataset >>>>> <MapDataset element_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None)>
아아아 tf.Tensor(b'\xed\x95\xad\xec\xa0\x95\xec\x82\xb4', shape=(), dtype=string)
아아아 tf.Tensor(b'\xeb\xb0\xa5', shape=(), dtype=string)
아아아 tf.Tensor(b'\xea\xb9\x8d\xeb\x91\x90\xea\xb8\xb0', shape=(), dtype=string)
아아아 tf.Tensor(b'\xec\x8c\x80\xeb\xb0\xa5', shape=(), dtype=string)
아아아 tf.Tensor(b'\xec\x82\xbc\xea\xb2\xb9\xec\x82\xb4', shape=(), dtype=string)
아아아 tf.Tensor(b'\xea\xb9\x80\xec\xb9\x98\xeb\xa7\x8c\xeb\x91\x90', shape=(), dtype=string)
아아아 tf.Tensor(b'\xeb\x9d\xbc\xeb\xa9\xb4, \xeb\xb9\x84\xeb\xb9\x94\xeb\x9d\xbc\xeb\xa9\xb4, \xeb\x81\x93\xec\x9d\xb8\xea\xb2\x83', shape=(), dtype=string)
아아아 tf.Tensor(b'\xeb\x90\x9c\xec\x9e\xa5\xec\xb0\x8c\xea\xb0\x9c', shape=(), dtype=string)
아아아 tf.Tensor(b'\xec\xbb\xb5\xeb\x9d\xbc\xeb\xa9\xb4', shape=(), dtype=string)
아아아 tf.Tensor(b'\xeb\x96\xa1\xeb\xb3\xb6\xec\x9d\xb4', shape=(), dtype=string)
아아아 tf.Tensor(b'\xeb\xb6\x88\xea\xb3\xa0\

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/Trainer/model/12/Format-Serving/assets


INFO:absl:Training complete. Model written to /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/Trainer/model/12/Format-Serving. ModelRun written to /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/Trainer/model_run/12
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 12
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Exporting the model

After training the model, we can use the [Pusher component](https://www.tensorflow.org/tfx/guide/pusher) to export the model.

In [56]:
_serving_model_dir = './test2'
PIPELINE_NAME = 'testt'
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)))
context.run(pusher, enable_cache=True)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1677168713
INFO:absl:Model written to serving path serving_model/testt/1677168713.
INFO:absl:Model pushed to /tmp/tfx-interactive-2023-02-23T16_07_49.631911-mnzbfulq/Pusher/pushed_model/13.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 13
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

## Make predictions

Now that we have a model, we load it back and make predictions.

## Next step

In this tutorial, you have learned how to implement a retrieval model with TensorFlow Recommenders and TFX. To expand on what is presented here, have a look at the [TFRS ranking with TFX](https://www.tensorflow.org/recommenders/examples/ranking_tfx) tutorial.